## 必要なプログラムパッケージのインストール

まず、Pythonにpyscf, qulacs, openfermion, openfermion-pyscfをインストールする。(qulacsのインストールにはgcc7以上が必要。)
```
pip install pyscf openfermion
pip install openfermionpyscf
pip install qulacs
```

その後JuliaやPyCall.jlの導入。。。。
この他にもscipyやmatplotlibをこのノートでは使用している。

In [ ]:
import Revise

In [ ]:
#import QCMaterial: uccgsd, convert_openfermion_op, up_index, down_index, update_circuit_param!, update_quantum_state!
using QCMaterial
using PyCall
scipy_opt = pyimport("scipy.optimize")

## 2-site Hubbard modelのハミルトニアンを定義する。

$H=-t \sum_{\langle i, j>\sigma=(\uparrow, \downarrow)} \sum_{i, \sigma}\left(a_{i, \sigma}^{\dagger} a_{j, \sigma}+a_{j, \sigma}^{\dagger} a_{i, \sigma}\right)+U \sum_{i} n_{i, \uparrow} n_{i, \downarrow}-\mu \sum_{i} \sum_{\sigma=(\uparrow, \downarrow)} n_{i, \sigma}$

$1_\uparrow$->0,$1_\downarrow$->1, $2_\uparrow$->2, $2_\downarrow$->3

クーロン斥力(U=2): 2 * $n_{1,\uparrow}$ $n_{1,\downarrow}$ + 2 * $n_{2,\uparrow}$ $n_{2,\downarrow}$ 
               = 2 * $c^\dagger_0$ $c_0$ $c^\dagger_1$ $c_1$ + 2 * $c^\dagger_2$ $c_2$ $c^\dagger_3$ $c_3$
 
ホッピング項(t=-0.1): -0.1 * $c^\dagger_{1,\uparrow}$ $c_{2,\uparrow}$ -0.1 * $c^\dagger_{2,\uparrow}$ $c_{1,\uparrow}$
                   -0.1 * $c^\dagger_{1,\downarrow}$ $c_{2,\downarrow}$ -0.1 * $c^\dagger_{2,\downarrow}$ $c_{1,\downarrow}$
  = -0.1 * $c^\dagger_0$ $c_2$ + -0.1 * $c^\dagger_2$ $c_0$ + -0.1 * $c^\dagger_1$ $c_3$ + -0.1 * $c_3^\dagger$ $c_1$

ケミカルポテンシャル項(μ=U/2=1):-1 * $n_{1,\uparrow}$ - 1 * $n_{1,\downarrow}$ - 1 * $n_{2,\uparrow}$ - 1 * $n_{2,\downarrow}$ 
               = - 1 * $c^\dagger_0$ $c_0$ - 1 * $c^\dagger_1$ $c_1$ - 1 * $c^\dagger_2$ $c_2$ - 1 * $c^\dagger_3$ $c_3$

In [ ]:
#ハバードモデルハミルトニアンを定義し、対角化まで行う
nsite = 2 #サイトの数
n_qubit = 2*nsite #量子ビットの数
U = 0.0
t = -2.0

#ハーフフィリングを仮定(電子数 = サイトの数)するとケミカルポテンシャルはμ=U/2.マイナスがつくので以下。
μ = U/2

ham = FermionOperator()
#斥力項
for i in 1:nsite
    #up_index,down_indexの定義は、QC_materialを参照。
    up = up_index(i)
    down = down_index(i)
    ham += FermionOperator("$(up)^ $(down)^ $(up) $(down)", -U) #左側に生成演算子。右際に消滅演算子をもっていく過程で半交換関係が1回でマイナスをつける。
end
#ホッピング項
for i in 1:nsite-1
    ham += FermionOperator("$(up_index(i+1))^ $(up_index(i))", t) 
    ham += FermionOperator("$(up_index(i))^ $(up_index(i+1))", t) 
    ham += FermionOperator("$(down_index(i+1))^ $(down_index(i))", t) 
    ham += FermionOperator("$(down_index(i))^ $(down_index(i+1))", t) 
end

#ケミカルポテンシャルの項
for i in 1:nsite
    up = up_index(i)
    down = down_index(i)
    ham += FermionOperator("$(up)^  $(up) ", -μ) 
    ham += FermionOperator("$(down)^ $(down)", -μ)
end

println(ham)　#ハバードモデルハミルトニアンの表示。

n_electron = 2　#電子数。ここではハーフフィリングを仮定しているので、電子数はサイトの数と等しくなければならない。
@assert mod(n_electron, 2) == 0
sparse_mat = get_number_preserving_sparse_operator(ham, n_qubit, n_electron)　#行列の取得

using LinearAlgebra
enes_ed = eigvals(sparse_mat.toarray())　#対角化を行う
enes_ed

In [ ]:
#最小固有値
EigVal_min = minimum(enes_ed)
EigVal_min

## Jordan-Wigner展開
2サイトのハバードモデルのハミルトニアンを用意し、スピン系にマッピングする。
ここではJordan-Wigner展開を用いることとする。

In [ ]:
#JW変換後、量子回路で計測可能なObservableに変換
ham_obs = create_observable(jordan_wigner(ham), n_qubit)

## Unitary Coupled Cluster (UCC)の概略

Hartree-Fock波動関数を $|\Phi \rangle$ とすると、Unitary Coupled Cluster（UCC)は以下の形で定義される。
$$|\Psi \rangle =e^{\hat{T}-\hat{T}^{\dagger}}|\Phi \rangle$$
これに対して通常のCoupled Cluster（CC）は以下の形を取る。
$$|\Psi \rangle =e^{\hat{T}}|\Phi \rangle$$
最も広く使われるCC or UCCは $\hat{T}$ を1電子励起 $\hat{T}_1$ と2電子励起 $\hat{T}_2$ に限定したCCSD or UCCSDとなっている。
CCSDないしUCCSDの $\hat{T}$ は具体的には次のように定義される。
$$
\hat{T} = \sum_{ai}t^{a}_{i} a^{\dagger}_a a_i + 
1/4\sum_{ab,ij}t^{ab}_{ij} a^{\dagger}_a a^{\dagger}_b a_j a_i
$$ 
ここで添字 $a,b$ は仮想軌道を示し、 $i,j$ は占有軌道を示す。


CCはHamiltonian $\hat{H}$ のsimilarity transformation $e^{-\hat{T}}\hat{H}e^{T}$ のBCH展開が打ち切れるため古典コンピュータで実装可能である。
一方で、UCCは脱励起演算子 $\hat{T}^{\dagger}$ のためにBCH展開が打ちきれず無限に続くために古典コンピュータでは計算不可能であるが、
$e^{\hat{T}-\hat{T}^{\dagger}}$ はユニタリー変換であるため量子回路で記述できる。
ただし、 $e^{\hat{T}-\hat{T}^{\dagger}}$ を量子回路で記述するためにはこのユニタリー演算子をTrotter分解しなくてはならない。
しかし、 $T$ の各タームは可換ではないため量子コンピュータでのUCCの実装はトロッター誤差が生じる。
ただし、このトロッター誤差はvariationalなアルゴリズムを用いる場合は変数決定の過程で概ね吸収できることがわかっている。
そこで実用上まずはトロッターステップを１とした次式が使われる。
$$
e^{\hat{T}-\hat{T}^{\dagger}} \approx
\prod^{n_{occ}}_{i>j} \prod^{n_{vir}}_{a>b}
\{
e^{T^{ab}_{ij} a^\dagger_a a^\dagger_b a_j a_i - 
T^{ab*}_{ij} a^\dagger_i a^\dagger_j a_b a_a}
\}
\prod^{n_{occ}}_{i} \prod^{n_{vir}}_{a} 
\{
e^{T^{a}_{i} a^\dagger_a a_i -T^{a*}_{i} a^\dagger_i a_a }
\}
$$
ここで ${n_{occ}}$ は占有軌道の数を、 $n_{vir}$ は仮想軌道の数を示す。



## 軌道の最適化
軌道の回転を行う。軌道の回転とは、以下の$\kappa$に関する波動関数の最適化に対応する。（論文（Orbital optimized unitary coupled cluster theory for quantum computer）(4)式より）
$$
E(A, \kappa)=\left\langle\Psi\left|e^{-\hat{\kappa}} \hat{H} e^{\hat{\kappa}}\right| \Psi\right\rangle=\left\langle 0\left|e^{-\hat{A}} e^{-\hat{\kappa}} \hat{H} e^{\hat{k}} e^{\hat{A}}\right| 0\right\rangle
$$

$$
\hat{\kappa}=\sum_{pq}\kappa_{pq}(\hat{E_{pq}}-\hat{E_{qp}})
$$
ここで $\hat{E}_{p q}=\hat{c}_{p, \alpha}^{\dagger} \hat{c}_{q, \alpha}+\hat{c}_{p, \beta}^{\dagger} \hat{c}_{q, \beta}$

2サイトのハバードモデルの場合、$\hat{\kappa}=\kappa_02()



UCCSDのサーキットを用意し、VQE計算に必要なエネルギーを計算するための関数（cost関数）を用意する。cost関数をscipyのoptimizerにセットすることで、VQEのパラメータ決定が可能になる（この次のセルで実行）。

In [ ]:
circuit = uccgsd(n_qubit, true)
println("Number of Qubits:", n_qubit)
println("Number of Electrons:", n_electron)
function cost(theta_list)
    state = create_hf_state(n_qubit, n_electron)
    update_circuit_param!(circuit, theta_list) 
    update_quantum_state!(circuit, state) 
    get_expectation_value(ham_obs, state) 
end

In [ ]:
theta_init = rand(num_theta(circuit))
cost_history = Float64[] #コスト関数の箱
init_theta_list = theta_init
push!(cost_history, cost(init_theta_list))
println("cost_history", cost_history)

method = "BFGS"
options = Dict("disp" => true, "maxiter" => 50, "gtol" => 1e-5)
callback(x) = push!(cost_history, cost(x))
opt = scipy_opt.minimize(cost, init_theta_list, method=method, callback=callback)
println("cost_history=",cost_history)

VQE計算を実行。

VQEの収束状況をプロットする。

In [ ]:
import PyPlot
PyPlot.plot(cost_history, color="red", label="VQE")
PyPlot.plot(1:length(cost_history), fill(EigVal_min, length(cost_history)),
    linestyle="dashed", color="black", label="Exact Solution")
PyPlot.xlabel("Iteration")
PyPlot.ylabel("Energy expectation value")
PyPlot.legend()
PyPlot.show()

In [ ]:
divide_real_imag_openfermion(op) = 
    (op+hermitian_conjugated(op))/2, (op-hermitian_conjugated(op))/2im

function make_rdm1(state::QuantumState, n_qubit)
    one_rdm = zeros(Complex{Float64}, n_qubit, n_qubit)
    for p in 1:n_qubit, q in 1:n_qubit
        # Use a vector create a Fermion operator (not Tuple)
        jw_hamiltonian = jordan_wigner(FermionOperator([(p-1, 1), (q-1, 0)]))
        her, antiher = divide_real_imag_openfermion(jw_hamiltonian)
        qulacs_hamiltonian_real = create_observable(her, n_qubit)
        qulacs_hamiltonian_imag = create_observable(antiher, n_qubit)
        pq_one_rdm_real = get_expectation_value(qulacs_hamiltonian_real, state)
        pq_one_rdm_imag = get_expectation_value(qulacs_hamiltonian_imag, state)
        one_rdm[p, q] = pq_one_rdm_real + pq_one_rdm_imag * im
    end
    one_rdm
end

## 縮約密度行列の計算法
以下のコードブロックでは、VQE波動関数から１次の縮約密度行列
$$
\gamma_{pq} =
\langle \Psi |
a^{\dagger}_p a_q
|\Psi \rangle 
$$
を計算する方法を示す。

In [ ]:
import LinearAlgebra

state = create_hf_state(n_qubit, n_electron)
rdm_hf = make_rdm1(state, n_qubit)
println("RDM of HF")
println(rdm_hf)
println("Number of electrons (HF)")
println(real(LinearAlgebra.tr(rdm_hf)))

update_quantum_state!(circuit, state) #量子回路を状態に作用
rdm_uccsd = make_rdm1(state, n_qubit)
println("RDM of UCCSD")
println(rdm_uccsd)
println("Number of electrons (UCCSD)")
println(real(LinearAlgebra.tr(rdm_hf)))    